In [1]:
import sqlite3
import pandas as pd

# connect to the database
conn = sqlite3.connect(":memory:")

# create the cursor
cursor = conn.cursor()

# open and run the .sql commands
# data source UDACITY:
# https://video.udacity-data.com/topher/2020/May/5eb5533b_parch-and-posey/parch-and-posey.sql
sql_file = open("parch-and-posey.sql")
sql_as_string = sql_file.read()
cursor.executescript(sql_as_string)

# Let's see the schema of the database

In [5]:
# show the schema
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('web_events',), ('sales_reps',), ('region',), ('orders',), ('accounts',)]


In [10]:
# run a query
pd.read_sql('SELECT * FROM web_events LIMIT 5', conn)

id  account_id          occurred_at channel
0   1        1001  2015-10-06 17:13:58  direct
1   2        1001  2015-11-05 03:08:26  direct
2   3        1001  2015-12-04 03:57:24  direct
3   4        1001  2016-01-02 00:55:03  direct
4   5        1001  2016-02-01 19:02:33  direct

In [11]:
pd.read_sql('SELECT * FROM sales_reps LIMIT 5', conn)

id             name  region_id
0  321500    Samuel Racine          1
1  321510     Eugena Esser          1
2  321520  Michel Averette          1
3  321530    Renetta Carew          1
4  321540      Cara Clarke          1

In [12]:
pd.read_sql('SELECT * FROM region LIMIT 5', conn)

id       name
0   1  Northeast
1   2    Midwest
2   3  Southeast
3   4       West

In [13]:
pd.read_sql('SELECT * FROM orders LIMIT 5', conn)

id  account_id          occurred_at  standard_qty  gloss_qty  poster_qty  \
0   1        1001  2015-10-06 17:31:14           123         22          24   
1   2        1001  2015-11-05 03:34:33           190         41          57   
2   3        1001  2015-12-04 04:21:55            85         47           0   
3   4        1001  2016-01-02 01:18:24           144         32           0   
4   5        1001  2016-02-01 19:27:27           108         29          28   

   total  standard_amt_usd  gloss_amt_usd  poster_amt_usd  total_amt_usd  
0    169            613.77         164.78          194.88         973.43  
1    288            948.10         307.09          462.84        1718.03  
2    132            424.15         352.03            0.00         776.18  
3    176            718.56         239.68            0.00         958.24  
4    165            538.92         217.21          227.36         983.49

In [14]:
pd.read_sql('SELECT * FROM accounts LIMIT 5', conn)

id                name                    website        lat       long  \
0  1001             Walmart            www.walmart.com  40.238496 -75.103297   
1  1011         Exxon Mobil         www.exxonmobil.com  41.169156 -73.849374   
2  1021               Apple              www.apple.com  42.290495 -76.084009   
3  1031  Berkshire Hathaway  www.berkshirehathaway.com  40.949021 -75.763898   
4  1041            McKesson           www.mckesson.com  42.217093 -75.284998   

      primary_poc  sales_rep_id  
0     Tamara Tuma        321500  
1    Sung Shields        321510  
2      Jodee Lupo        321520  
3  Serafina Banda        321530  
4  Angeles Crusoe        321540

We can see that we have 5 tables and we can also see the primary and foreign keys of each table

# Transform the data

Running this query we can know which clients have spent more than 200,000, between 200k and 100k and less than 100k

In [21]:
query = """SELECT a.name, SUM(o.total_amt_usd) total_spent,
                 CASE
                 WHEN SUM(o.total_amt_usd) > 200000 THEN 'Greater than 200,000'
                 WHEN SUM(o.total_amt_usd) > 100000 AND SUM(o.total_amt_usd) < 200000 THEN '200,000 and 100,000'
                 ELSE 'under 100,000' END AS level
                 FROM orders o
                 JOIN accounts a
                 ON o.account_id = a.id
                 GROUP BY 1
                 ORDER BY 2 DESC;"""

pd.read_sql(query, conn)

name  total_spent                 level
0             EOG Resources    382873.30  Greater than 200,000
1                    Mosaic    345618.59  Greater than 200,000
2                       IBM    326819.48  Greater than 200,000
3          General Dynamics    300694.79  Greater than 200,000
4         Republic Services    293861.14  Greater than 200,000
..                      ...          ...                   ...
345       Bed Bath & Beyond      1069.64         under 100,000
346                   Deere      1036.57         under 100,000
347  Level 3 Communications       881.73         under 100,000
348         Delta Air Lines       859.64         under 100,000
349                    Nike       390.25         under 100,000

[350 rows x 3 columns]

Running this query we can know which channels were the most used to contact the clients

In [25]:
query = """SELECT web_events.channel, COUNT(accounts.name)
 FROM web_events
 JOIN accounts
 ON web_events.account_id = accounts.id
 GROUP BY web_events.channel
 ORDER BY COUNT(accounts.name) DESC
 LIMIT 8;"""

pd.read_sql(query, conn)

channel  COUNT(accounts.name)
0    direct                  5298
1  facebook                   967
2   organic                   952
3   adwords                   906
4    banner                   476
5   twitter                   474

Running this query we can see how many times each client was contacted using "facebook"

In [29]:
query = """
SELECT accounts.name, COUNT(web_events.channel)
 FROM accounts
 JOIN web_events
 ON web_events.account_id = accounts.id
 WHERE web_events.channel LIKE 'facebook'
 GROUP BY accounts.name
 HAVING COUNT(web_events.channel) > 6
 ORDER BY 2
 LIMIT 10;"""

pd.read_sql(query, conn)

name  COUNT(web_events.channel)
0            Avis Budget Group                          7
1                     Best Buy                          7
2   Farmers Insurance Exchange                          7
3                   Home Depot                          7
4      Honeywell International                          7
5            J.P. Morgan Chase                          7
6  Laboratory Corp. of America                          7
7            Leucadia National                          7
8              Parker-Hannifin                          7
9                  Wells Fargo                          7

Running this query we can see how many times each channel was used in each region

In [36]:
query = """
SELECT region.name, web_events.channel, COUNT(web_events.channel) number_of_times
 FROM web_events
 JOIN accounts
 ON accounts.id = web_events.account_id
 JOIN sales_reps
 ON sales_reps.id = accounts.sales_rep_id
 JOIN region
 ON sales_reps.region_id = region.id
 GROUP BY region.name, web_events.channel
 ORDER BY 3 DESC
 LIMIT 10;
 """

pd.read_sql(query, conn)

name   channel  number_of_times
0  Northeast    direct             1800
1  Southeast    direct             1548
2       West    direct             1254
3    Midwest    direct              696
4  Northeast  facebook              335
5  Northeast   organic              317
6  Northeast   adwords              300
7  Southeast  facebook              278
8  Southeast   organic              275
9  Southeast   adwords              264

# Loading

Then we load any query to a .CSV file

In [39]:
pd.read_sql(query, conn).to_csv('times_per_region_channel.csv',index=False)